<a href="https://colab.research.google.com/github/gurusakharwade/LP5-HPC/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ruqrof3v
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ruqrof3v
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=57ae5d2aa464a902dd5655c07e56c23ecbc3d8bc9b9dc848dab29f0d95559c7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ti75e5nw/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [ ]:
#cuda Max Reduce
%%cu
#include<stdio.h>
#include<cuda.h>
#include<stdlib.h>
#include<time.h>

__global__ void max1(int* input)
{
    const int tid = threadIdx.x;

    auto step_size = 1;
    int number_of_threads = blockDim.x;
    int temp;

    while (number_of_threads > 0)
    {
        if (tid < number_of_threads) // still alive?
        {
            const auto fst = tid * step_size * 2;
            const auto snd = fst + step_size;
            if (input[fst] < input[snd])
            {
                temp = input[fst];
                input[fst] = input[snd];
                input[snd] = temp;
            }
        }
        __syncthreads();
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

int main()
{
    const auto count = 8;
    const int size = count * sizeof(int);
    int h[] = {13, 65, 15, 14, 33, 2, 30, 8};

    int* d;

    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    max1 <<<1, count / 2 >>>(d);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
    printf("Largest number is %d\n", result);

    cudaFree(d);

    return 0;
}


Largest number is 65



In [ ]:
 #cuda arrayAdd Reduce
 %%cu
 #include<stdio.h>
 #include<cuda.h>
 #include<stdlib.h>
 #include<time.h>
 
 __global__ void sum(int* input)
 {
 	const int tid = threadIdx.x;
 
 	auto step_size = 1;
 	int number_of_threads = blockDim.x;
 
 	while (number_of_threads > 0)
 	{
 		if (tid < number_of_threads) // still alive?
 		{
 			const auto fst = tid * step_size * 2;
 			const auto snd = fst + step_size;
 			input[fst] += input[snd];
 		}
 		__syncthreads();
 		step_size <<= 1; 
 		number_of_threads >>= 1;
 	}
 }
 
 int main()
 {
 	const auto count = 8;
 	const int size = count * sizeof(int);
 	int h[] = {13, 27, 15, 14, 33, 2, 30, 8};
 
 	int* d;
 	
 	cudaMalloc(&d, size);
 	cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);
 
 	sum <<<1, count / 2 >>>(d);
 
 	int result;
 	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
  	// cout << "Sum is " << result << endl;
  printf("Sum is %d  ", result);
 
 	getchar();
 
 	cudaFree(d);
 	//delete[] h;
 
 	return 0;
 }


Sum is 142  


In [ ]:
 #cude min reduce
 %%cu
 #include<stdio.h>
 #include<cuda.h>
 #include<stdlib.h>
 #include<time.h>
 
 __global__ void min1(int* input)
 {
 	const int tid = threadIdx.x;
 
 	auto step_size = 1;
 	int number_of_threads = blockDim.x;
  int temp;
 
 	while (number_of_threads > 0)
 	{
 		if (tid < number_of_threads) // still alive?
 		{
 			const auto fst = tid * step_size * 2;
 			const auto snd = fst + step_size;
 			//input[fst] += input[snd];
       if (input[fst]>input[snd])
       {
           temp=input[fst];
           input[fst]=input[snd];
           input[snd]=temp;
       }
 		}
 		__syncthreads();
 		step_size <<= 1; 
 		number_of_threads >>= 1;
 	}
 }
 
 int main()
 {
 	const auto count = 8;
 	const int size = count * sizeof(int);
 	int h[] = {13, 65, 15, 14, 33, 23, 30, 8};
 
 	int* d;

 	cudaMalloc(&d, size);
 	cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);
 
 	min1 <<<1, count / 2 >>>(d);
 
 	int result;
 	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
  	// cout << "Large no is %d " << result << endl;
  printf("Small no is %d  ", result);
 
 	getchar();
 
 	cudaFree(d);
 	//delete[] h;
 
 	return 0;
 }


Small no is 8  
